The decision tree graph is drawn for the puzzle given for 3x3. The goal node is indicated in green.

In [ ]:
from graphviz import Digraph
import os
import copy
import functools
import time
import random


counter = 1
#initial = [random.sample(range(3), 3) , random.sample(range(3,6), 3) , random.sample(range(9), 3)]
print("please enter n for n-puzzle")
n = int(input())
initial= [random.sample(range(n), n)...random.sample(range(n*(n-1), n*n), n)]
print(initial)

initial =[[0, 1, 3,9],
          [6, 2, 4,11],
          [7, 5, 8,12],
         [13,10,14,15]]


#final =[random.sample(range(3), 3) , random.sample(range(3,6), 3) , random.sample(range(9), 3)]
final = [[1, 2, 3],
         [6, 5, 4],
         [7, 8, 0]]


class State(object):
    def __init__(self, self_state, parent_id, parent_move, level): # parent_move is the move that led to this state
        global counter # counter is used to assign unique id to each state
        self.self_id = counter # id of this state
        self.self_state = self_state # state of this state
        self.parent_id = parent_id # id of parent state
        self.parent_move = parent_move # move that led to this state
        self.level = level # level of this state
        self.cost = 0 # cost of this state
        self.l_child = -1 # id of left child
        self.u_child = -1 # id of up child
        self.r_child = -1 # id of right child
        self.d_child = -1 # id of down child
        counter += 1 # increment counter

    def set_child(self, state_id, move): # set child of this state
        if move == "left":
            self.l_child = state_id
        elif move == "right":
            self.r_child = state_id
        elif move == "up":
            self.u_child = state_id
        elif move == "down":
            self.d_child = state_id

    def __repr__(self): # print state
        return str(self.self_state[0])+"\n"+str(self.self_state[1])+"\n"+str(self.self_state[2])


def create_child(parent_state, move):
    i = 0 # row index of 0
    j = 0 # column index of 0
    found = False # flag to check if 0 is found
    matrix = copy.deepcopy(parent_state) # copy parent state
    for i in range(0, 3): # find 0
        for j in range(0, 3):
            if parent_state[i][j] == 0: # if 0 is found
                found = True # set flag to true
                break
        if found:
            break
    if move == "left":
        if j == 0:
            return -1
        else:
            matrix[i][j] = matrix[i][j - 1] # swap 0 and left element
            matrix[i][j - 1] = 0 # swap 0 and left element
            return matrix # return new state
    if move == "right":
        if j == 2:
            return -1
        else:
            matrix[i][j] = matrix[i][j + 1] # swap 0 and right element
            matrix[i][j + 1] = 0 # swap 0 and right element
            return matrix
    if move == "up":
        if i == 0:
            return -1
        else:
            matrix[i][j] = matrix[i - 1][j] # swap 0 and up element
            matrix[i - 1][j] = 0 # swap 0 and up element
            return matrix

    if move == "down":
        if i == 2:
            return -1
        else:
            matrix[i][j] = matrix[i + 1][j] # swap 0 and down element
            matrix[i + 1][j] = 0 # swap 0 and down element
            return matrix


explored = []
unexplored = []

def custom_sort(a, b): # sort function for priority queue
    if a.cost == b.cost: # if cost is same
        return 0
    elif a.cost < b.cost: # if cost of a is less than cost of b
        return 1
    else:
        return -1

def manhattan_distance(current_matrix): # calculate manhattan distance
    goal_matrix = [[1,2,3],
                [6,5,4],
                [7,8,0]]
    distance = 0
    for i in range(0,3):
        for j in range(0,3):
            if current_matrix[i][j] != 0: # if element is not 0
                for item in goal_matrix: # find element in goal matrix
                    if current_matrix[i][j] in item: # if element is found
                        x = goal_matrix.index(item)  # row index of element in goal matrix
                        y = item.index(current_matrix[i][j]) # column index of element in goal matrix
                distance = distance + (abs(x - i) + abs(y - j)) # calculate manhattan distance
    return distance


def A_star(): # A* algorithm
    global state_space_tree, explored, unexplored # state_space_tree is the tree of all states
    global id_to_state
    current_state = id_to_state[1] # current state is initial state
    counter_g = 0 # counter for number of nodes expanded

    dot = Digraph(comment='BFS graph') # create graph
    dot.node(str(current_state.self_id), current_state.__str__()) # add initial state to graph

    while True:
        explored.append(current_state)
        # print(current_state.self_id, len(explored))
        # # graph
        # dot.node(str(current_state.self_id), current_state.__str__())
        # dot.edge(str(current_state.parent_id), str(current_state.self_id), label=current_state.parent_move)
        if current_state.self_state == final:
            break # if current state is final state, break
        if current_state.u_child != -1:
            child_state = id_to_state[current_state.u_child] # get up child
            cost_child = counter_g + manhattan_distance(child_state.self_state) # calculate cost of child
            child_state.cost = cost_child # set cost of child
            if child_state not in explored and child_state not in unexplored: # if child is not explored or unexplored
                unexplored.append(child_state) # add child to unexplored
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move)
        if current_state.d_child != -1: # same as above
            child_state = id_to_state[current_state.d_child]
            cost_child = counter_g + manhattan_distance(child_state.self_state)
            child_state.cost = cost_child
            if child_state not in explored and child_state not in unexplored:
                unexplored.append(child_state)
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move)
        if current_state.l_child != -1:
            child_state = id_to_state[current_state.l_child] # get left child
            cost_child = counter_g + manhattan_distance(child_state.self_state) # calculate cost of child
            child_state.cost = cost_child
            if child_state not in explored and child_state not in unexplored: # if child is not explored or unexplored
                unexplored.append(child_state) # add child to unexplored
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move)
        if current_state.r_child != -1: # same as above
            child_state = id_to_state[current_state.r_child]
            cost_child = counter_g + manhattan_distance(child_state.self_state)
            child_state.cost = cost_child
            if child_state not in explored and child_state not in unexplored:
                unexplored.append(child_state)
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move)

        unexplored.sort(key=lambda x:x.cost)

        current_state = unexplored.pop(0)
        print([item.self_id for item in unexplored]) # print unexplored nodes
        print([item.cost for item in unexplored]) # print cost of unexplored nodes
        print("\n")
        # time.sleep(1)
        counter_g += 1

    dot.render(str(os.getcwd() + '/outputs/A_star_graph.gv'), view=True) # render graph


def Greedy_best_first():
    global state_space_tree, explored, unexplored # state_space_tree is the tree of all states
    global id_to_state
    current_state = id_to_state[1] # current state is initial state

    explored = []
    unexplored = []

    dot = Digraph(comment='BFS graph') # create graph
    dot.node(str(current_state.self_id), current_state.__str__()) # add initial state to graph

    while True:
        explored.append(current_state)
        # print(current_state.self_id, len(explored))
        # # graph
        # dot.node(str(current_state.self_id), current_state.__str__())
        # dot.edge(str(current_state.parent_id), str(current_state.self_id), label=current_state.parent_move)
        if current_state.self_state == final:
            break
        if current_state.u_child != -1:
            child_state = id_to_state[current_state.u_child] # get up child
            cost_child = manhattan_distance(child_state.self_state) # calculate cost of child
            child_state.cost = cost_child # set cost of child
            if child_state not in explored and child_state not in unexplored:
                unexplored.append(child_state)
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move+str(child_state.cost))
        if current_state.d_child != -1:
            child_state = id_to_state[current_state.d_child]
            cost_child = manhattan_distance(child_state.self_state)
            child_state.cost = cost_child
            if child_state not in explored and child_state not in unexplored:
                unexplored.append(child_state)
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move+str(child_state.cost))
        if current_state.l_child != -1:
            child_state = id_to_state[current_state.l_child] # get left child
            cost_child = manhattan_distance(child_state.self_state)
            child_state.cost = cost_child
            if child_state not in explored and child_state not in unexplored:
                unexplored.append(child_state)
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move+str(child_state.cost))
        if current_state.r_child != -1:
            child_state = id_to_state[current_state.r_child]
            cost_child = manhattan_distance(child_state.self_state)
            child_state.cost = cost_child
            if child_state not in explored and child_state not in unexplored:
                unexplored.append(child_state)
                # graph
                dot.node(str(child_state.self_id), child_state.__str__())
                dot.edge(str(current_state.self_id), str(child_state.self_id), label=child_state.parent_move+str(child_state.cost))

        unexplored.sort(key=lambda x:x.cost)

        current_state = unexplored.pop(0)

    dot.render(str(os.getcwd() + '/outputs/greedy_best_first_graph.gv'), view=True)

state_space_tree = dict()
id_to_state = dict()

if __name__ == "__main__":
    # for graph
    dot = Digraph(comment='State space graph')

    opposite_movements = {"up": "down", "down": "up", "left": "right", "right": "left"} # opposite movements
    goal_found = False  # flag to check if goal is found
    current_state = State(initial, 1, "null", 1)  # initial state
    id_to_state[1] = current_state
    explore_queue = [current_state.self_id] # queue to explore

    dot.node(str(current_state.self_id), current_state.__str__())   # add initial state to graph

    goal_lvl = 9999 # goal level

    while (True):
        current_state = id_to_state[explore_queue.pop(0)] # pop first element from queue

        if current_state.level  > goal_lvl + 1: # if level of current state is greater than goal level + 1, break
            break

        parent_move = current_state.parent_move # get parent move
        possible_movements = ["up", "left", "down", "right"]    # possible movements
        if (parent_move != "null"):
            possible_movements.remove(opposite_movements[parent_move]) # remove opposite movement
        childs = []
        for move in possible_movements: # for each possible movement
            matrix = create_child(current_state.self_state, move) # create child
            if matrix != -1:
                state_obj = State(matrix, current_state.self_id, move, current_state.level+1) # create state object
                current_state.set_child(state_obj.self_id, move) # set child
                # put the child id to queue
                explore_queue.append(state_obj.self_id) # add child to queue
                # save the child object
                id_to_state[state_obj.self_id] = state_obj
                childs.append(state_obj.self_id)
                # check if this child is the goal state
                if matrix == final and goal_lvl == 9999:
                    goal_lvl = state_obj.level - 1

                if matrix == final:
                    dot.node(str(state_obj.self_id), state_obj.__str__(), color='green') # add goal state to graph
                else:
                    dot.node(str(state_obj.self_id), state_obj.__str__()) # add child to graph

                dot.edge(str(state_obj.parent_id), str(state_obj.self_id), label=move) # add edge to graph

    A_star()
    Greedy_best_first()
    dot.render(str(os.getcwd() + '/outputs/state_space_tree_informed.gv'), view=True) # render graph


In [3]:
from graphviz import Digraph
import os
import copy

counter = 1


class State(object): # state class

    def __init__(self, self_state, parent_id, parent_move, level): # state initialization
        global counter
        self.self_id = counter
        self.self_state = self_state
        self.parent_id = parent_id
        self.parent_move = parent_move
        self.level = level
        self.l_child = -1
        self.u_child = -1
        self.r_child = -1
        self.d_child = -1
        counter += 1

    def set_child(self, state_id, move): # set child
        if move == "left":
            self.l_child = state_id
        elif move == "right":
            self.r_child = state_id
        elif move == "up":
            self.u_child = state_id
        elif move == "down":
            self.d_child = state_id

    def __repr__(self):
        return str(self.self_state[0])+"\n"+str(self.self_state[1])+"\n"+str(self.self_state[2])


def create_child(parent_state, move): # create child
    i = 0
    j = 0
    found = False
    matrix = copy.deepcopy(parent_state)
    for i in range(0, 3):
        for j in range(0, 3):
            if parent_state[i][j] == 0:
                found = True
                break
        if found:
            break
    if move == "left":
        if j == 0:
            return -1
        else:
            matrix[i][j] = matrix[i][j - 1]
            matrix[i][j - 1] = 0
            return matrix
    if move == "right":
        if j == 2:
            return -1
        else:
            matrix[i][j] = matrix[i][j + 1]
            matrix[i][j + 1] = 0
            return matrix
    if move == "up":
        if i == 0:
            return -1
        else:
            matrix[i][j] = matrix[i - 1][j]
            matrix[i - 1][j] = 0
            return matrix

    if move == "down":
        if i == 2:
            return -1
        else:
            matrix[i][j] = matrix[i + 1][j]
            matrix[i + 1][j] = 0
            return matrix


def bfs_search(): # bfs search
    global state_space_tree
    global id_to_state
    parent_state = id_to_state[1]

    dot = Digraph(comment='BFS graph')
    dot.node(str(parent_state.self_id), parent_state.__str__()) # add parent node to graph

    queue = [parent_state] # add parent state to queue
    while(len(queue) != 0): # while queue is not empty
        current_state = queue.pop(0) # pop first element from queue
        if current_state.u_child != -1: # if up child exists
            child_state = id_to_state[current_state.u_child] # get child state
            queue.append(child_state) # add child state to queue
            # graph
            dot.node(str(child_state.self_id), child_state.__str__()) # add child node to graph
            dot.edge(str(current_state.self_id), str(child_state.self_id), label="up") # add edge to graph
            if child_state.self_state == final_state: # if child state is goal state
                break
        if current_state.l_child != -1: # if left child exists
            child_state = id_to_state[current_state.l_child] # get child state
            queue.append(child_state) # add child state to queue
            # graph
            dot.node(str(child_state.self_id), child_state.__str__()) # add child node to graph
            dot.edge(str(current_state.self_id), str(child_state.self_id), label="left") # add edge to graph
            if child_state.self_state == final_state: # if child state is goal state
                break
        if current_state.d_child != -1:
            child_state = id_to_state[current_state.d_child] # get child state
            queue.append(child_state)
            # graph
            dot.node(str(child_state.self_id), child_state.__str__()) # add child node to graph
            dot.edge(str(current_state.self_id), str(child_state.self_id), label="down") # add edge to graph
            if child_state.self_state == final_state: # if child state is goal state
                break
        if current_state.r_child != -1:
            child_state = id_to_state[current_state.r_child] # get child state
            queue.append(child_state)
            # graph
            dot.node(str(child_state.self_id), child_state.__str__()) # add child node to graph
            dot.edge(str(current_state.self_id), str(child_state.self_id), label="right")
            if child_state.self_state == final_state: # if child state is goal state
                break

    dot.render(str(os.getcwd() + '/outputs/BFS_graph.gv'), view=True) # render graph


initial_state = [[0, 1, 3],
                [6, 2, 4],
                [7, 5, 8]]

final_state = [[1, 2, 3],
                [6, 5, 4],
                [7, 8, 0]]
visited = []

goal_found = False

dot_dfs = Digraph(comment='DFS graph')

def dfs(node): # dfs search
    global visited
    global dot_dfs
    global goal_found
    if goal_found:
        return
    if node not in visited:
        visited.append(node)

        dot_dfs.node(str(node.self_id), node.__str__())
        dot_dfs.edge(str(node.parent_id), str(node.self_id), label=node.parent_move)

        if node.self_state == final_state: # if child state is goal state
            goal_found = True
            return
        if node.u_child != -1:
            child_state = id_to_state[node.u_child]
            dfs(child_state)
        if node.l_child != -1:
            child_state = id_to_state[node.l_child]
            dfs(child_state)
        if node.d_child != -1:
            child_state = id_to_state[node.d_child]
            dfs(child_state)
        if node.r_child != -1:
            child_state = id_to_state[node.r_child]
            dfs(child_state)


state_space_tree = dict()
id_to_state = dict() # dictionary to store id to state mapping

if __name__ == "__main__":
    # state_space_tree
    # global id_to_state
    # for graph
    dot = Digraph(comment='State space graph') # create graph

    opposite_movements = {"up": "down", "down": "up", "left": "right", "right": "left"}
    goal_found = False # flag to check if goal is found
    current_state = State(initial_state, 1, "null", 1)
    id_to_state[1] = current_state
    explore_queue = [current_state.self_id]

    dot.node(str(current_state.self_id), current_state.__str__())

    goal_lvl = 9999

    while (True):
        current_state = id_to_state[explore_queue.pop(0)]

        if current_state.level  > goal_lvl + 1:
            break

        parent_move = current_state.parent_move # get parent move
        possible_movements = ["up", "left", "down", "right"]
        if (parent_move != "null"):
            possible_movements.remove(opposite_movements[parent_move]) # remove opposite movement
        childs = []
        for move in possible_movements:
            matrix = create_child(current_state.self_state, move) # create child state
            if matrix != -1:
                state_obj = State(matrix, current_state.self_id, move, current_state.level+1) # create child state
                current_state.set_child(state_obj.self_id, move)
                # put the child id to queue
                explore_queue.append(state_obj.self_id)
                # save the child object
                id_to_state[state_obj.self_id] = state_obj # save child state object
                childs.append(state_obj.self_id) # add child id to childs list
                # check if this child is the goal state
                if matrix == final_state and goal_lvl == 9999:
                    goal_lvl = state_obj.level - 1

                if matrix == final_state:
                    dot.node(str(state_obj.self_id), state_obj.__str__(), color='green')
                else:
                    dot.node(str(state_obj.self_id), state_obj.__str__())

                dot.edge(str(state_obj.parent_id), str(state_obj.self_id), label=move) # add edge to graph
    dot.render(str(os.getcwd() + '/outputs/state_space_tree.gv'), view=True) # render graph

    bfs_search()

    parent = id_to_state[1] # get root node
    dfs(parent)
    dot_dfs.render(str(os.getcwd() + '/outputs/DFS_graph.gv'), view=True)